In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def get_value(values):
    if len(values)>1:
        print('more than 1 value!')
        return -1
    elif len(values)==0:
        return 0
    else:
        return values[0]

In [ ]:
clas6_data = pd.read_csv('clas6/CLAS6.txt', delim_whitespace=True)
clas6_data.head()

In [ ]:
clas6_tra = pd.read_csv('clas6/clas6_Tra_unpol_all.txt',delim_whitespace=True)
clas6_tra.columns=['SF_Set_Index','Eb','beam_pol','target_pol','x','y','z','ph_t_sq','phih','phi_s','k0','bn','amm','nrad_ir','rad_f','rad_f_err','nrad','nrad_err','rad','rad_err','nrad_rad','nrad_rad_err']
clas6_tra.head()

In [ ]:
clas6_fac_bn = pd.read_csv('clas6/clas6_Fac_bn_unpol_all.txt',delim_whitespace=True)
clas6_fac_bn.columns=['x','z','Q','qT_Q','xs','xs_err']
clas6_fac_bn.head()

In [ ]:
clas6_fac_rc = pd.read_csv('clas6/clas6_Fac_rc_unpol_all.txt',delim_whitespace=True)
clas6_fac_rc.columns=['x','z','Q','qT_Q','xs','xs_err']
clas6_fac_rc.head()

In [ ]:
M_P = 0.9382720813
beam_energy = 5.75
sqrt_s = math.sqrt(2*M_P*beam_energy)

In [ ]:
#To combine data, fac, and tra results
file = open('clas6/clas6_all.txt','w')
file.write('Q2,x,z,pT2,phi,data_xs,data_xs_err,data_xs_sys,data_xs_RC,tra_bnxs,tra_rcxs,tra_rcxs_err,Fac_bnxs,Fac_rcxs,Fac_rcxs_err\n')
for i,row in clas6_data.iterrows():
    #for tra
    Eb = beam_energy
    beam_pol ='U'
    target_pol = 'U'
    x = row['x']
    Q2 = (row['Q2'])
    s = 2*Eb*M_P
    y = round(Q2/(s*x),6)
    z = row['z']
    pT2 = row['pT2']
    ph_t_sq = row['pT2']
    phih=round((row['phi']-180)*math.pi/180,6)
    #for fac
    Q = round(math.sqrt(row['Q2']),6)
    qT_Q = round(math.sqrt(row['pT2'])/z/Q,6)
    
    xs_data = row['ds_dxdQ2dzdpT2dphi']*2.5/1000
    bnxs_tra = get_value(clas6_tra[(clas6_tra['x']==x)&(clas6_tra['y']==y)&(clas6_tra['z']==z)&(clas6_tra['ph_t_sq']==ph_t_sq)&(clas6_tra['phih']==phih)]['bn'].values)
    bnxs_fac = 1/(2*math.pi)*get_value(clas6_fac_bn[(clas6_fac_bn['x']==x)&(clas6_fac_bn['z']==z)&(clas6_fac_bn['Q']==Q)&(clas6_fac_bn['qT_Q']==qT_Q)]['xs'].values)
    #print('data',xs_data)
    #print('tra',bnxs_tra)
    #print('fac',bnxs_fac)
    xs_data_err = row['stat_err']*2.5/1000
    xs_data_sys_err = row['sys_err']*2.5/1000
    xs_data_rc = row['rad_cor']
    rcxs_tra = get_value(clas6_tra[(clas6_tra['x']==x)&(clas6_tra['y']==y)&(clas6_tra['z']==z)&(clas6_tra['ph_t_sq']==ph_t_sq)&(clas6_tra['phih']==phih)]['rad'].values)
    rcxs_tra_err = get_value(clas6_tra[(clas6_tra['x']==x)&(clas6_tra['y']==y)&(clas6_tra['z']==z)&(clas6_tra['ph_t_sq']==ph_t_sq)&(clas6_tra['phih']==phih)]['rad_err'].values)
    rcxs_fac = 1/(2*math.pi)*get_value(clas6_fac_rc[(clas6_fac_rc['x']==x)&(clas6_fac_rc['z']==z)&(clas6_fac_rc['Q']==Q)&(clas6_fac_rc['qT_Q']==qT_Q)]['xs'].values)
    rcxs_fac_err = 1/(2*math.pi)*get_value(clas6_fac_rc[(clas6_fac_rc['x']==x)&(clas6_fac_rc['z']==z)&(clas6_fac_rc['Q']==Q)&(clas6_fac_rc['qT_Q']==qT_Q)]['xs_err'].values)
    print('xs_data_err',xs_data_err,'xs_data_sys_err',xs_data_sys_err,'xs_data_rc',xs_data_rc)
    print('rcxs_tra',rcxs_tra,'rcxs_tra_err',rcxs_tra_err,'rcxs_fac',rcxs_fac,'rcxs_fac_err',rcxs_fac_err)
    file.write('%f,%f,%f,%f,%f,%E,%E,%E,%E,%E,%E,%E,%E,%E,%E\n'%(Q2,x,z,pT2,phih,xs_data,xs_data_err,xs_data_sys_err,xs_data_rc,bnxs_tra,rcxs_tra,rcxs_tra_err,bnxs_fac,rcxs_fac,rcxs_fac_err))

